<a href="https://colab.research.google.com/github/shashank-yadav/RNN-sharpe/blob/master/rnn_sharpe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function, division
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

num_epochs = 100
total_series_length = 50000
truncated_backprop_length = 15
state_size = 4
num_classes = 1
echo_step = 3
batch_size = 5
num_batches = total_series_length//batch_size//truncated_backprop_length

In [0]:
def generateData():
    x = np.array(np.random.choice(2, total_series_length, p=[0.5, 0.5]))
    y = np.roll(x, echo_step)
    y[0:echo_step] = 0

    x = x.reshape((batch_size, -1))  # The first index changing slowest, subseries as rows
    y = y.reshape((batch_size, -1))

    return (x, y)

In [3]:
x,y = generateData()
print(x.shape, y.shape)

(5, 10000) (5, 10000)


In [0]:
batchX_placeholder = tf.placeholder(tf.float32, [batch_size, truncated_backprop_length])
batchY_placeholder = tf.placeholder(tf.int32, [batch_size, truncated_backprop_length])

init_state = tf.placeholder(tf.float32, [batch_size, state_size])

In [0]:
# Unpack columns
# inputs_series = tf.unstack(batchX_placeholder, axis=1) # list of length truncated_backprop_length with each element as a tensor of size batch
inputs_series = tf.split(batchX_placeholder, truncated_backprop_length, axis=1)
labels_series = tf.unstack(batchY_placeholder, axis=1)

In [6]:
cell = tf.nn.rnn_cell.BasicRNNCell(state_size)
states_series, current_state = tf.nn.static_rnn(cell, inputs_series, dtype=tf.float32)

W2 = tf.Variable(np.random.rand(state_size, num_classes),dtype=tf.float32)
b2 = tf.Variable(np.zeros((1,num_classes)), dtype=tf.float32)
logits_series = [tf.matmul(state, W2) + b2 for state in states_series]


Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API
Instructions for updating:
Colocations handled automatically by placer.


In [0]:
def sharpe(y):
    y_stacked = tf.stack(y, axis=1)
    mean = tf.math.reduce_mean(y_stacked, axis=1)
    std = tf.math.reduce_std(y_stacked, axis=1)
    return mean/std

loss = -sharpe(logits_series)
train_step = tf.train.AdagradOptimizer(0.005).minimize(loss)

In [11]:

with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    for epoch_idx in range(num_epochs):
        x,y = generateData()
        _current_state = np.zeros((batch_size, state_size))

        print("New data, epoch", epoch_idx)

        for batch_idx in range(num_batches):
            start_idx = batch_idx * truncated_backprop_length
            end_idx = start_idx + truncated_backprop_length

            batchX = x[:,start_idx:end_idx]
            batchY = y[:,start_idx:end_idx]

            _, out_loss = sess.run(
                [train_step, loss],
                feed_dict={
                    batchX_placeholder:batchX
#                     batchY_placeholder:batchY
                })

            
            if batch_idx%100 == 0:
                print("Step",batch_idx, "Loss", out_loss)

New data, epoch 0
Step 0 Loss [[0.7404966 ]
 [0.6060658 ]
 [0.6581357 ]
 [0.23070641]
 [0.5861331 ]]
Step 100 Loss [[-379.11548]
 [-156.35622]
 [-158.47792]
 [-385.0404 ]
 [-159.26361]]
Step 200 Loss [[-471677.84  ]
 [  -6851.8755]
 [  -6848.6426]
 [  -6851.8755]
 [-470452.2   ]]
Step 300 Loss [[nan]
 [nan]
 [nan]
 [nan]
 [nan]]
Step 400 Loss [[nan]
 [nan]
 [nan]
 [nan]
 [nan]]
Step 500 Loss [[nan]
 [nan]
 [nan]
 [nan]
 [nan]]
Step 600 Loss [[nan]
 [nan]
 [nan]
 [nan]
 [nan]]
New data, epoch 1
Step 0 Loss [[nan]
 [nan]
 [nan]
 [nan]
 [nan]]
Step 100 Loss [[nan]
 [nan]
 [nan]
 [nan]
 [nan]]
Step 200 Loss [[nan]
 [nan]
 [nan]
 [nan]
 [nan]]
Step 300 Loss [[nan]
 [nan]
 [nan]
 [nan]
 [nan]]
Step 400 Loss [[nan]
 [nan]
 [nan]
 [nan]
 [nan]]
Step 500 Loss [[nan]
 [nan]
 [nan]
 [nan]
 [nan]]
Step 600 Loss [[nan]
 [nan]
 [nan]
 [nan]
 [nan]]
New data, epoch 2
Step 0 Loss [[nan]
 [nan]
 [nan]
 [nan]
 [nan]]
Step 100 Loss [[nan]
 [nan]
 [nan]
 [nan]
 [nan]]
Step 200 Loss [[nan]
 [nan]
 [nan]
 [